In [42]:
from transformers import AutoTokenizer, AutoModel, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb-ctrl")

model = AutoModelWithLMHead.from_pretrained("lvwerra/gpt2-imdb-ctrl")

from fastai.text.all import *

Some weights of the model checkpoint at lvwerra/gpt2-imdb-ctrl were not used when initializing GPT2LMHeadModel: ['v_head.summary.weight', 'v_head.summary.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [43]:
def clean_html(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def complete_word_transformer(language_model, tokenizer, text, final_word):
    if len(text) == 0:
        return ''
    ids = tokenizer.encode(text)
    t = torch.LongTensor(ids)[None].to('cuda')
    logits = language_model.forward(t).logits[0][-1]
    sorted_indices = torch.argsort(logits, descending=True)
    
    for tk_idx in sorted_indices:
        word = tokenizer.decode(tk_idx.cpu().numpy())
        if word.lower().startswith(final_word):
            return word[len(final_word):]
    return ""

def generate_text_transformer(language_model, tokenizer, text, n_words_max):
    ids = tokenizer.encode(text)
    t = torch.LongTensor(ids)[None].to('cuda')
    phrase = language_model.generate(input_ids=t, num_beams=5, temperature=1.2,  max_length=(len(ids) + 10), skip_special_tokens=True, do_sample=False, repetition_penalty=2.)
#     phrase = language_model.generate(input_ids=t, n_words_max=20)
    prediction = phrase[0][t.size(1):].cpu()
    prediction = prediction[prediction!=50256]
    
    return clean_html(tokenizer.decode(prediction.cpu().numpy())[1:])

In [45]:
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [65]:
generate_text_transformer(model, tokenizer, prompt, n_words_max=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'than most of the other JR JR JR JR JR'

In [64]:
prompt = "[positive]The movie is one of Adam Sorkin's better movies. It tries to portray a relaistic depiction of love and romance while not shying away from the more awkward moments that often accompany the romantic experience.The acting is different"